## Import FEMA data for housing assistance

In [37]:
import pandas as pd
import requests

Below is the API URL provided by FEMA that lists disasters. Used a dummy header and pulled in the data using the requests.get method. Check the status code that it was successfull (should be 200).

In [ ]:
url = 'https://www.fema.gov/api/open/v1/HousingAssistanceOwners'
headers = {'User-agent': 'UserB 1.27'}
api = requests.get(url, headers=headers)
api.status_code

Convert the data into json type and look at the keys.

In [5]:
dj = api.json()
sorted(dj.keys())

Put the data into a dataframe

In [6]:
df = pd.DataFrame(dj['HousingAssistanceOwners'])

By using '?$skip=xxxx' you can loop through each 'page' of the api to retrieve all of the values. 1000 is the max number of entries you can retrieve per page.

In [7]:
var = 1000
for x in range(1000):
    url = f'https://www.fema.gov/api/open/v1/HousingAssistanceOwners?$skip={var}'
    try:
        api = requests.get(url, headers={'User-agent': 'UserB 1.27'})
        json = api.json()
        df_temp = pd.DataFrame(json['HousingAssistanceOwners'])
        df = pd.concat([df, df_temp], axis=0)
    except:
        print(f'failed{x}')
    var = var + 1000

Check the shape and if there are duplicates

In [10]:
df.shape

(90579, 26)

In [11]:
df.duplicated().sum()

0

### Clean the Data

In [ ]:
df.columns

Rename the columns

In [ ]:
new_cols= {
'approvedBetween10001And25000': 'approve_10001_25000', 
'approvedBetween1And10000': 'approve_1_10000',
'approvedBetween25001AndMax': 'approve_25001_max', 
'approvedForFemaAssistance': 'approve_assistance',
'averageFemaInspectedDamage': 'avg_damage', 
'disasterNumber': 'disaster_number',
'femaInspectedDamageBetween10001And20000': 'inspect_10001_20000',
'femaInspectedDamageBetween1And10000': 'inspect_1_10000',
'femaInspectedDamageBetween20001And30000': 'inspect_20001_30000',
'femaInspectedDamageGreaterThan30000': 'inspect_greater_30000', 
'lastRefresh': 'last_refresh',
'noFemaInspectedDamage': 'no_damage', 
'otherNeedsAmount': 'other_needs_amt', 
'rentalAmount': 'rental_amt',
'repairReplaceAmount': 'repair_replace_amt', 
'totalApprovedIhpAmount': 'tot_approve_ihp_amt', 
'totalDamage': 'tot_damage',
'totalInspected': 'tot_inspected', 
'totalMaxGrants': 'tot_max_grants', 
'validRegistrations': 'valid_registration', 
}

In [ ]:
df.rename(columns=new_cols, inplace=True)

Reorder the column names

In [ ]:
df = df[[
'disaster_number',
'state',   
'county', 
'city',
'zipCode',
'valid_registration', 
'avg_damage',
'tot_inspected',
'tot_damage',
'no_damage',
'inspect_1_10000', 
'inspect_10001_20000',  
'inspect_20001_30000',
'inspect_greater_30000',
'approve_assistance', 
'tot_approve_ihp_amt',
'repair_replace_amt', 
'rental_amt', 
'other_needs_amt',
'approve_1_10000',     
'approve_10001_25000', 
'approve_25001_max',
'tot_max_grants',
'hash', 
'id', 
'last_refresh',  
]]

Drop columns that were created in the pull and not related to the data.

In [ ]:
df.drop(columns=['hash', 'id', 'last_refresh'], inplace=True)

### Export to csv

In [34]:
df.to_csv('./data/fema_clean.csv', index=False)